In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os

In [ ]:
# cap = cv2.VideoCapture("lisa.mp4")
cap = cv2.VideoCapture(0)
#Pose
mpPose = mp.solutions.pose
pose = mpPose.Pose()
myDraw = mp.solutions.drawing_utils

# Hands
mpHand = mp.solutions.hands
hands = mpHand.Hands()
myDraw = mp.solutions.drawing_utils

#Face Mesh
myDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faces = mpFaceMesh.FaceMesh(max_num_faces = 32)
draw = myDraw.DrawingSpec(thickness = 1,circle_radius=1)

#Load forder image
forderpath = "image_finger/"
myLs=os.listdir(forderpath)
overlaylist = []
for i in myLs:
    image = cv2.imread(f"{forderpath}{i}")
    overlaylist.append(image)
tipIdx = [4,8,12,16,20]
cTime = 0
pTime = 0
while True:
    ref,img = cap.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
    
    
    #Draw pose
    if results.pose_landmarks:
        myDraw.draw_landmarks(img,results.pose_landmarks,mpPose.POSE_CONNECTIONS)
        for id,land in enumerate(results.pose_landmarks.landmark):
            h,w,c = img.shape # high,weight,chanel with img
            cx = int(land.x *w)
            cy = int(land.y *h)
            cv2.circle(img,(cx,cy),8,(233,127,0),cv2.FILLED)
            
    #Draw hands
    lst = []
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            myDraw.draw_landmarks(img,hand,mpHand.HAND_CONNECTIONS)
            for id,la in enumerate(hand.landmark):
                h,w,c = img.shape
                cx,cy = int(la.x*w),int(la.y*h)
                lst.append([id,cx,cy])
              
                cv2.circle(img,(cx,cy),6,(255,127,0),cv2.FILLED)
            
    #Draw face
    results = faces.process(imgRGB)
    if results.multi_face_landmarks:
        for face in results.multi_face_landmarks:
            myDraw.draw_landmarks(img,face,mpFaceMesh.FACEMESH_CONTOURS,draw,draw)
    #Count number
    if len(lst) != 0:
        finger = []
        #Thumb
        if lst[tipIdx[0]][1] > lst[tipIdx[0]-1][1]:
            finger.append(1)
        else:
            finger.append(0)
        #4 finger
        for idd in range(1,5):
            if lst[tipIdx[idd]][2] < lst[tipIdx[idd]-2][2]:
                finger.append(1)
            else:
                finger.append(0)
        total = finger.count(1)
        cv2.putText(img,str(total),(10,400),cv2.FONT_HERSHEY_PLAIN,10,(255,255,5),2,cv2.LINE_AA)

        h,w,c = overlaylist[total].shape
        img[0:h,0:w] = overlaylist[total]       
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img,"FPS: " +str(int(fps)),(10,50),cv2.FONT_HERSHEY_PLAIN,2,(2,127,120),2,cv2.LINE_AA)
    cv2.imshow("Video",img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
            break
cv2.destroyAllWindows()